# 保存与加载

[![下载Notebook](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_notebook.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/master/tutorials/zh_cn/beginner/mindspore_save_load.ipynb)&emsp;
[![下载样例代码](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_download_code.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/master/tutorials/zh_cn/beginner/mindspore_save_load.py)&emsp;
[![查看源文件](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_source.png)](https://gitee.com/mindspore/docs/blob/master/tutorials/source_zh_cn/beginner/save_load.ipynb)

上一章节的内容里面主要是介绍了如何调整超参数，并进行网络模型训练。训练网络模型的过程中，实际上我们希望保存中间和最后的结果，用于微调（fine-tune）和后续的模型部署和推理，本章节我们开始学习如何保存与加载模型。

## 保存模型

我们使用MindSpore提供的save_checkpoint保存模型，传入网络和保存路径：


In [1]:
import mindspore as ms
import mindspore.nn as nn
from mindspore.common.initializer import Normal

# 定义神经网络
class LeNet(nn.Cell):
    def __init__(self, num_classes=10, num_channel=1):
        super(LeNet, self).__init__()
        layers = [nn.Conv2d(num_channel, 6, 5, pad_mode='valid'),
                  nn.ReLU(),
                  nn.MaxPool2d(kernel_size=2, stride=2),
                  nn.Conv2d(6, 16, 5, pad_mode='valid'),
                  nn.ReLU(),
                  nn.MaxPool2d(kernel_size=2, stride=2),
                  nn.Flatten(),
                  nn.Dense(16 * 5 * 5, 120, weight_init=Normal(0.02)),
                  nn.ReLU(),
                  nn.Dense(120, 84, weight_init=Normal(0.02)),
                  nn.ReLU(),
                  nn.Dense(84, num_classes, weight_init=Normal(0.02))]
        # 使用CellList对网络进行管理
        self.build_block = nn.CellList(layers)

    def construct(self, x):
        # for循环执行网络
        for layer in self.build_block:
            x = layer(x)
        return x

# 实例化训练网络
network = LeNet()

# 保存模型的参数到指定路径
ms.save_checkpoint(network, "./MyNet.ckpt")

- `save_checkpoint`方法会把网络的参数保存到指定路径中。其中，`network`为训练网络，`"./MyNet.ckpt"`为网络模型的保存路径。

## 加载模型

要加载模型权重，需要先创建相同模型的实例，然后使用`load_checkpoint`和`load_param_into_net`方法加载参数。

示例代码如下：

In [ ]:
import mindspore as ms

# 将模型参数存入parameter的字典中，这里加载的是上面训练过程中保存的模型参数
param_dict = ms.load_checkpoint("./MyNet.ckpt")

# 重新定义一个LeNet神经网络
net = LeNet()

# 将参数加载到网络中并打印未加载的参数名称
not_loaded_params = ms.load_param_into_net(net, param_dict)
print(not_loaded_params)

[]


- `load_checkpoint`方法会把参数文件中的网络参数加载到字典`param_dict`中。
- `load_param_into_net`方法会把字典`param_dict`中的参数加载到网络或者优化器中，加载后，网络中的参数就是CheckPoint保存的。
